### Importing libraries

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import random # library for random number generation

!pip install geopy
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from IPython.display import Image 
from IPython.core.display import HTML 

!conda install --override-channels -c main package_in_main

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - package_in_main

Current channels:

  - https://conda.anaconda.org/main/linux-64
  - https://conda.anaconda.org/main/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


Libraries imported.


### Foursquare data

In [7]:
CLIENT_ID = '0ZLRMIPIQ1MNB4IROWDSKAIZTRD3JAGVOTHDUR3YTCIGVKXS' # your Foursquare ID
CLIENT_SECRET = 'BWJHQAANUAO1AVG45F4LZUIL3MCJ1CTKMAMUNMIMUTT5AWOA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0ZLRMIPIQ1MNB4IROWDSKAIZTRD3JAGVOTHDUR3YTCIGVKXS
CLIENT_SECRET:BWJHQAANUAO1AVG45F4LZUIL3MCJ1CTKMAMUNMIMUTT5AWOA


### Define Coordinares of the Center location

In [8]:
geolocator = Nominatim (user_agent="foursquare_agent")
latitude = 52.4454164
longitude = 13.5752346
categoryId = '52f2ab2ebcbc57f1066b8b46'
radius = 1000
print(latitude, longitude)

52.4454164 13.5752346


### Define the URL 

In [9]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&latitude={}&longitude={}&categoryId={}&radius={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude,
    categoryId, 
    radius)
results = requests.get(url).json()

### Transform the Data into a Pandas Dataframe 

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
                   
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results["response"]['venues']
#venues = results["response"]['groups'][0]['items']
# flatten JSON to venues
nearby_venues = json_normalize(venues)

# keep only columns that include venue name, and anything that is associated with location
#filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng'] - send option from coursera 
filtered_columns = ['name', 'categories'] + [col for col in nearby_venues.columns if col.startswith('location.')] + ['id']
nearby_venues1 = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1)
nearby_venues1.columns = [col.split(".")[-1] for col in nearby_venues1.columns]
nearby_venues1.drop(columns=['crossStreet', 'address','labeledLatLngs','postalCode', 'city','state','country', 'formattedAddress','id','cc','categories'], inplace = True)

nearby_venues1

#example with pizza

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

       # filter the category for each row
       dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
       # clean column names by keeping only last term
      dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered.drop(columns=['crossStreet', 'address','labeledLatLngs','postalCode', 'city','state','country', 'formattedAddress','id','cc','categories'], inplace = True)
dataframe_filtered

In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues1.shape[0]))

NameError: name 'nearby_venues1' is not defined

### Create the Map

In [ ]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 
# generate map centred around the Rathaus in Koepenick
# add a red circle marker to represent the central point - Rathaus in Koepenick
folium.CircleMarker(
       [latitude+0.001, longitude-0.0025],
    radius=20,
    color='blue',
    popup='Rathaus in Koepenick',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(nearby_venues1.lat, nearby_venues1.lng, nearby_venues1.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=folium.Popup(label, parse_html=True),
        fill = True,
        fill_color='blue',
        fill_opacity=0.6).add_to(venues_map)
   
    display(venues_map)
            

### K-means

In [ ]:
lockers_grouped = nearby_venues1.groupby('name').mean().reset_index()
lockers_grouped_clustering = lockers_grouped.drop('name', 1)

In [ ]:
# set number of clusters
kclusters = 3
#manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lockers_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
lockers_grouped_clustering.insert(0, 'Cluster Labels', kmeans.labels_)
lockers_grouped_clustering

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)\nys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lockers_grouped['lat'], lockers_grouped['lng'], lockers_grouped['name'], lockers_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters